# [LEGALST-190] Lab 3/13: Parsing XML Data

This lab will cover parsing XML and attribute lookup, XPath, and web scraping.

*Estimated Time: 45 Minutes *

### Topics Covered:
- XML syntax
- locating content with XPATH
- Web scraping

### Table of Contents
[The Data](#section data)<br>
1 - [XML Syntax](#section 1)<br>
2 - [Using XPath and ElementTree to parse XML](#section 2)<br>
3 - [Web Scraping](#section 3)<br>
4 - [Putting it all in a dataframe](#section 4)<br>

**Dependencies:**

In [1]:
import pandas as pd
import xml.etree.cElementTree as ET #XML Parser
from lxml import etree #ElementTree and lxml allow us to parse the XML file.
import requests #make request to server
import time #pause loop

----
## The Data<a id='section data'></a>

In this notebook, you'll be working with XML files from the Old Bailey API (https://www.oldbaileyonline.org/obapi/). These files contain the proceedings of all trials from 1674 to 1913. For this lab, we'll go through the trials from 1754-1756 and 1824-1826. XML (eXtensible Markup Language) provides a hierarchical representation of data contained within different tags and nodes. We'll go over XML syntax later. We will learn how to parse through these XML files from Old Bailey and grab information from sections of an XML file.

---

## Section 1: XML Syntax<a id='section 1'></a>

First, we'll go over the syntax of a XML file. The basic unit of XML code is called an "element" or "node" and has a start and ending tag. The tags for each element look something like this:

`<exampletag>some text</exampletag>`

Run the next cell to look at the XML file of one of the cases from the OldBailey API!

In [2]:
#For now, don't worry about the code for now, we'll go through it later.
example = requests.get('https://www.oldbaileyonline.org/obapi/text?div=t17031013-13')
print(example.text)

<?xml version="1.0" encoding="UTF-8"?>
<div1 type="trialAccount" id="t17031013-13">
               <interp inst="t17031013-13" type="collection" value="BAILEY"></interp>
               <interp inst="t17031013-13" type="year" value="1703"></interp>
               <interp inst="t17031013-13" type="uri" value="sessionsPapers/17031013"></interp>
               <interp inst="t17031013-13" type="date" value="17031013"></interp>
               <join result="criminalCharge" id="t17031013-13-off60-c52" targOrder="Y" targets="t17031013-13-defend52 t17031013-13-off60 t17031013-13-verdict64"></join>
         
               <p>
            
                  <persName id="t17031013-13-defend52" type="defendantName">
                  Samuel 
                  Davis
               <interp inst="t17031013-13-defend52" type="surname" value="Davis"></interp>
                     <interp inst="t17031013-13-defend52" type="given" value="Samuel"></interp>
                     <interp inst="t17031013-13-d

The `interp` tags at the beginning of the file are elements that don't have any plain text content. Note that elements may possibly be empty and not contain any text (i.e. `interp` elements mentioned earlier). If the element is empty, the tag may follow a format that looks similar to `<exampletag/>`, which is equivalent to `<exampletag></exampletag>`.

Elements may also contain other elements, which we call "children". Most children are indented, but the indents aren't necessary in XML and are used for clarity to show nesting. For example, if we go down to `<persName id="t17540116-4-defend46" type="defendantName">` , we see that the `rs` tag is a child of `persName`. We will explore about children in XML more in the next section. 

Lastly, elements may have attributes, which are in the format `<exampletag name_of_attribute="somevalue">`. Attributes are designed to store data related to a specific elements. Attributes **must** follow the quotes format (`name = "value"`). As you can tell, in this XML file, attributes are everywhere!

-----
**Question 1.1:** What was the verdict of this case? Was there a punsihment and if so, what was it? List both and state whether you found it as plain text content or as an attribute.

*Write your response here*

In [ ]:
#ANSWER
Verdict: guilty, plain text content
Punsihment: brandingOnCheek, attribute

----
## Section 2: Using XPath and `ElementTree` to parse XML<a id='section 2'></a>

Now that we know what the syntax and structure of an XML file, let's figure out how to parse through one! We are going to load the same file from the first section and use XPath (XML Path Language) to navigate through elements in this file. 

XPath is designed to locate content in an XML file and uses a ["tree" structure](https://www.researchgate.net/profile/Roger_Moussalli/publication/257631377/figure/fig8/AS:297441854279689@1447927072768/Example-XML-Document-and-XML-Path-Queries-a-Example-XML-Document-b-XML-Tree.png) to extract specific chunks. XPath expressions are made up of "location steps" which are separated by forward slashes.

First, we need to import the file into an ElementTree instance. The ElementTree format will allow us to go through each element, sorting through tags so we can extract the data we want.

In [3]:
xml_file = 'data/old-bailey-example.xml'
tree = ET.ElementTree(file=xml_file)
tree

We're going to start working from the root of the tree as XML files have a tree structure. Let's load the root of our tree. 

In [4]:
root = tree.getroot()
print(root)

<Element 'div1' at 0x111e1f9a8>


Now that we have the root, we can now start working down the tree! With the root, we can find each child of the root by printing the tags. This will also help us for future reference, if we every want to go through other children in the XML file.

In [5]:
#get child tags from root
for child in root:
    print(child.tag)

interp
interp
interp
interp
join
p
p


Now that we have a list of children to work with let's select one using `.find`. Using `.find` requires an XPath expression which will navigate through the hierarchical structure of XML and help us keep track of the path we are taking through this file.

In [6]:
choose_p = root.find('p')
for child in choose_p:
    print(child.tag)

persName
placeName
interp
interp
join
rs
persName
rs
join
rs
join
rs


This isn't very helpful, since we're still left with a bunch of tags and on top of that, we have a lot of repeating tags and names. Let's choose `placeName` as our next tag and see what happens. Notice that in our XPath expression, we are using foward slashes to navigate to the next child.

In [7]:
place_name = root.find('p/placeName')
for child in place_name:
    print(child.tag)

Looks like we hit the end! Let's use `.text` to examine the data in this element, following the `.find` path we used to get here.

In [8]:
print(root.find('p/placeName').text)
#alternatively, print(place_name.text)

St. James Westminster


Looking back at the file from earlier, we found where defendant was from. Let's see another feature of XPath we can utilize if, for instance, we know all of the possible children in the XML file. 

With XPath, you can either use a forward slash to move to the next element or child. So in our expression earlier, by following `'p/placeName`, we located any `placeName` element that is a child of `p`. Another way to navigate using XPath is using a period and a double forward slash (`.//`), which looks anywhere down the tree from your current element. So, if we start at the root and want to find any element with the tag `placeName`, we can do the following:

In [9]:
print(root.find('.//placeName').text)

St. James Westminster


**Experiment with this:** What happens if you don't have the period before the double slash? What happens if you change the starting element or use the whole XML file?

*Write your answer here*

**Question 2.1**: Find the defendant's name by traversing through the correct elements. You can check your answer in the printed XML file from [section 1](#section 1).

**Tip:** `print` your final expression so that it looks pretty!

In [ ]:
print(...) 

In [ ]:
#SOLUTION
print(tree.find('p/persName').text)

***WARNING*:** If you want to use `//` to find all elements with a specific child, you need to add a period (`.//`), since the node you're currently at most likely not absolute element ( the whole tree). If you want to try it out yourself, using `root.find('//placeName)` should give you an error but `root.find('.//placeName)` should give you what you want.

----
Luckily, we can use `.getiterator()`, a really helpful method from ElementTree. It creates an object which will let us iterate through all elements in the file. Using this method is powerful, as we can print each element name utilizing `.tag` or see the data for each element with `.text` and `.attrib`.

We can use `.getiterator()` on `tree`, our ElementTree instance. We call it in the form:

<p style="text-align: center;"> `tree.getiterator(tag=None)`  </p>

If you don't specify what tag you want, it'll go through the first element it comes across in `tree` and then through its children and their children, etc. If you only want elements with a specific tag name, like `placeName`, you can pass it as the argument.

Let's see how helpful `.getiterator()` can be! We'll call it on tree and print out the tag and attribute of each element.

In [10]:
iterator = tree.getiterator()
for element in iterator:
    print(element.tag)
    print(element.attrib)
    print()

div1
{'type': 'trialAccount', 'id': 't17031013-13'}

interp
{'inst': 't17031013-13', 'type': 'collection', 'value': 'BAILEY'}

interp
{'inst': 't17031013-13', 'type': 'year', 'value': '1703'}

interp
{'inst': 't17031013-13', 'type': 'uri', 'value': 'sessionsPapers/17031013'}

interp
{'inst': 't17031013-13', 'type': 'date', 'value': '17031013'}

join
{'result': 'criminalCharge', 'id': 't17031013-13-off60-c52', 'targOrder': 'Y', 'targets': 't17031013-13-defend52 t17031013-13-off60 t17031013-13-verdict64'}

p
{}

persName
{'id': 't17031013-13-defend52', 'type': 'defendantName'}

interp
{'inst': 't17031013-13-defend52', 'type': 'surname', 'value': 'Davis'}

interp
{'inst': 't17031013-13-defend52', 'type': 'given', 'value': 'Samuel'}

interp
{'inst': 't17031013-13-defend52', 'type': 'gender', 'value': 'male'}

placeName
{'id': 't17031013-13-defloc59'}

interp
{'inst': 't17031013-13-defloc59', 'type': 'placeName', 'value': 'St. James Westminster'}

interp
{'inst': 't17031013-13-defloc59', 't

**Question 2.2**: Using iterator and the information of the tags above, find the names of the defendant and the plaintiff by getting the text out of each element. You can either use a conditional to specify a tag and use `.tag` for some element, or specify a tag in `.getiterator()`.

***Note:*** Because of the formatting in the XML file, the you should only get the plaintiff's first name.

In [ ]:
...

In [11]:
#SOLUTION
for element in iterator:
    if element.tag == 'persName':
        print(element.text)


                  Samuel 
                  Davis
               

                  Catherine 
                  


What are their names? *Write their names in this cell*

**Question 2.3**: How do you think we can use `.attrib` to find their names? You don't have to code anything, just explain using words.

*Write your response here*  ***Should I make this optional?***

**Question 2.4:** Use `.getiterator()` again, and a new method, `.itertext()`, to get the entire text of the proceeding. Utilizing `.itertext()` method will return all inner text from every child.

**Hint:** Find the tag that will return you the entire text of the trial and a way to join all the text from the file together.

<sub>***Note:*** The text in these XML files are a little wonky, so if your printed text doesn't look nicely formatted, it's ok.</sub>

In [ ]:
for ... in ...:
    ...

In [12]:
for element in iterator:
    if element.tag == "p":
        print(''.join(list(element.itertext())))


            
                  
                  Samuel 
                  Davis
               
                     
                     
                  
            , of the Parish of St. James Westminster
                  
                  
                  , was indicted for 
                     
                     
               feloniously Stealing 58 Diamonds set in Silver gilt, value 250 l.
             the Goods of the Honourable 
               
                  Catherine 
                  Lady
                      
                  Herbert
               
                     
                     
                     
                  
            , on the 28th of July
                   last. It appeared that the Jewels were put up in a Closet, which was lockt, and the Prisoner being a Coachman
                   in the House, took his opportunity to take them; the Lady, when missing them, offered a Reward of Fourscore Pounds to any that could give any 

**Question 2.4:** Since the textual data is pretty messy in the XML files of these proceedings, where do you think the data you need might be held and how might you go about extracting this data? 

*Write your response here*

----
## Section 3: Web Scraping<a id='section 3'></a>

We learned how to get parse through one XML file. The Old Bailey API has a total of **197751** cases. Fortunately, we are only going to use the ones from 1754-1756 and 1824-1826, but that still only narrows are number of cases to 6506! 

Don't worry though, you're not going to have to manually download each case yourself. This is where web scraping comes into play. With web scraping, we can automate data collection to get all 6506 cases. 

Before we start scraping, we need to know how `requests` works. The `requests` library gets (`.get`!) you a response object from a web server and will automatically decode the content from the server, from which u can use `.text` to see the document! Requests through the Old Bailey API will return XML, which we can then write as a file and save.

Let's take a look at all of the terms we can use to choose the specific cases we want. We use `.json()` here since the parameters are stored as a JSON object.

In [ ]:
requests.get('http://www.oldbaileyonline.org/obapi/terms').json()

If you wanted to explore the full list in your web browser, click [this link](https://www.oldbaileyonline.org/obapi/terms). 

Now that you've had a chance to look through some of the terms, let's see how to grab the specific XML files.

Clicking the URL below returns a JSON object of the number of IDS and the frequency of each term in which every trial contains the term "sheffield" and the offence categrory "deception" from June 14th, 1847 onward. Also, each trial ID that satisfies the terms is returned; the count parameter in this case returns 10 trial IDs, but if left unspecified, the API will return a maximum count of 1000 IDs. 

https://www.oldbaileyonline.org/obapi/ob?term0=trialtext_sheffield&term1=offcat_deception&term2=fromdate_18470614&breakdown=offsubcat&count=10&start=0

Although the terms for time are listed as numbers, the format for the term is
`fromdate_(starting date)` and `todate_(ending date)` without the parentheses.

**Question 3.1:** Use requests.get(...) to get the all trial IDs between the years 1754 and 1756 and return it as a JSON object.

In [ ]:
trials = ...
trials

In [17]:
#SOLUTION 
trials = requests.get('https://www.oldbaileyonline.org/obapi/ob?term0=fromdate_17540116&term1=todate_17561208&&start=0').json()
trials

Now, lets pick some trials from `trial['hits']`, so we have a list of IDs we can work with. 

**Question 3.2:** Select the first 10 trials by splicing through the list.

In [ ]:
first_10 = ...
first_10

In [18]:
#SOLUTION
first_10 = trials['hits'][:10]
first_10

['t17540116-1',
 't17540116-2',
 't17540116-3',
 't17540116-4',
 't17540116-5',
 't17540116-6',
 't17540116-7',
 't17540116-8',
 't17540116-9',
 't17540116-10']

Using the trial IDs from the previous cell, we are going to format the URL in a way so that we can get the XML file for each trial. The way the Old Bailey API works in order to get the XML is through this URL format:

<p style="text-align: center;">`http://www.oldbaileyonline.org/obapi/text?div=(enter trial ID here without parenthesis)`  </p>

For example, http://www.oldbaileyonline.org/obapi/text?div=t16740429-1 give you the link to the XML file of the first proceeding in the database.


**Question  3.3:** Get the XML file of the first trial in first_10. You'll successfully get a response if your `.get` request returns `<Response [200]>`.

In [ ]:
...

In [19]:
#SOLUTION
url = 'http://www.oldbaileyonline.org/obapi/text?div={}'.format(first_10[0])
response = requests.get(url)
response

<Response [200]>

Run the next cell to see the XML format of the text! 

In [25]:
print(response.text)

<?xml version="1.0" encoding="UTF-8"?>
<div1 type="trialAccount" id="t17540116-1">
               <interp inst="t17540116-1" type="collection" value="BAILEY"></interp>
               <interp inst="t17540116-1" type="year" value="1754"></interp>
               <interp inst="t17540116-1" type="uri" value="sessionsPapers/17540116"></interp>
               <interp inst="t17540116-1" type="date" value="17540116"></interp>
               <join result="criminalCharge" id="t17540116-1-off2-c29" targOrder="Y" targets="t17540116-1-defend30 t17540116-1-off2 t17540116-1-verdict4"></join>
         
               <p>80. 
               
                  <persName id="t17540116-1-defend30" type="defendantName">
                     Hannah 
                     Ash 
                  <interp inst="t17540116-1-defend30" type="surname" value="Ash"></interp>
                     <interp inst="t17540116-1-defend30" type="given" value="Hannah"></interp>
                     <interp inst="t17540116-1-defe

We can save the XML file:

In [ ]:
trial_number = 't17540116-1' #trial ID (make sure its a string)
with open('data/old-bailey-' + trial_number + '.xml', 'w') as file:
    file.write(response.text)

### Challenge: Scraping all trials from 1754 - 1756

Now that you know how to find the trial IDs for certain parameters as well as get an XML file using `requests.get(some_url)`, iterate through each ID in the list of trials (use `trials['hits']` for the list of IDs) we got from 1754-1756 earlier. You can choose how many trials you want to save.

In [ ]:
#SOLUTION
for trial in ...:
    #format URL
    
    #get text from URL
    
    #save the file **store in data/old-bailey/file_name
    
    #one second pause so servers aren't overloaded
    
    

In [21]:
#SOLUTION
for trial in trials['hits'][:10]:
    #format URL
    url =  'http://www.oldbaileyonline.org/obapi/text?div={}'.format(trial)
    print(url)
    #get text from URL
    text = requests.get(url).text
    #save the file
    with open('data/old-bailey/old-bailey-' + trial + '.xml', 'w') as file:
        file.write(text)
    #one second pause so servers aren't overloaded
    time.sleep(1)

http://www.oldbaileyonline.org/obapi/text?div=t17540116-1
http://www.oldbaileyonline.org/obapi/text?div=t17540116-2
http://www.oldbaileyonline.org/obapi/text?div=t17540116-3
http://www.oldbaileyonline.org/obapi/text?div=t17540116-4
http://www.oldbaileyonline.org/obapi/text?div=t17540116-5
http://www.oldbaileyonline.org/obapi/text?div=t17540116-6
http://www.oldbaileyonline.org/obapi/text?div=t17540116-7
http://www.oldbaileyonline.org/obapi/text?div=t17540116-8
http://www.oldbaileyonline.org/obapi/text?div=t17540116-9
http://www.oldbaileyonline.org/obapi/text?div=t17540116-10


You can check if you saved the XML files by executing the cell below!

In [ ]:
!ls data/old-bailey/

This cell will show you the XML file.

In [ ]:
!cat data/old-bailey/old-bailey-t17540116-1.xml

----
## Section 4: Putting it all in a dataframe<a id='section 4'></a>

Now that we have a bunch of XML files and know how to parse through them to extract data, let's put the data from the XML files into a dataframe. As you probably saw earlier from printing the text of the court proceeding, the text was incredibly messy. Feel free to process the text yourself, but specifically for this last part, we'll use the data from each attribute to put in our dataframe.

**Question 4.1:** Complete the body of a function `table_of_cases`, which returns a dataframe with the "type" of data as a column label and the value from that attribute in that column. Make sure to account for cases that either won't have as many attributes as others (e.g. there are two defendants in one trial, but only one in the other). The body of the function is structured for you.

**Tips:** Open up different trials to see all "type" keys in attributes. How will you account for repeating "type" keys showing up repeatedly (e.g. surname, given, etc.) so that you don't replace the value you already have in the existing column with the same key? 

In [ ]:
def table_of_cases(xml_file):
    #load file
    
    #create an iterator object
    
    #create empty dataframe
    
    #create a possible index for repeating "types"
    
    for ... in ...:
        if ... == ...:
            #get attrib
    
            #get value of type
        
            #labels of columns in table
            
            #change possible index to string
            
            #Conditional clause 1
            
            #conditional clause 2
            
            #conditional clause 3
    
    return ...

In [22]:
#SOLUTION
def table_of_cases(xml_file):
    file = ET.ElementTree(file = xml_file)
    iterate = file.getiterator()
    i = 1
    table = pd.DataFrame()
    for element in iterate:
        if element.tag == "interp":
            t = element.attrib['type']
            val = [element.attrib['value']]
            labels = list(table.columns.values)
            num = str(i)
            if t not in labels:
                table[t] = val
            elif t+num not in labels:
                table[t+num] = val
            elif t+num in labels:
                num = str(i+1)
                table[t+num] = val
    return table

**Question 4.2:** Now, use `table_of_cases` to load the attribute data from each XML file that you scraped. Load a blank dataframe so you can append the table of information after each call.

In [ ]:
...

In [24]:
#SOLUTION
table = pd.DataFrame()
for i in trials['hits'][:10]:
    raw_data = 'data/old-bailey/old-bailey-'+ i +'.xml'
    data = table_of_cases(raw_data)
    table = table.append(data, ignore_index=True)
table

,collection,date,gender,gender1,gender2,given,given1,given2,offenceCategory,offenceCategory1,...,surname,surname1,surname2,type,uri,verdictCategory,verdictCategory1,verdictSubcategory,verdictSubcategory1,year
0,BAILEY,17540116,female,male,NaN,Hannah,Richard,NaN,theft,NaN,...,Ash,Beach,NaN,NaN,sessionsPapers/17540116,guilty,NaN,pleadedGuilty,NaN,1754
1,BAILEY,17540116,male,female,male,Peter,Mary,Joseph,theft,NaN,...,Foreman,Foreman,Sheers,NaN,sessionsPapers/17540116,guilty,NaN,NaN,NaN,1754
2,BAILEY,17540116,female,male,female,Sarah,Joseph,Ann,theft,NaN,...,Williams,Smithson,Smithson,crimeLocation,sessionsPapers/17540116,guilty,NaN,NaN,NaN,1754
3,BAILEY,17540116,male,female,male,Joseph,Elizabeth,Thomas,theft,NaN,...,Kempster,Kempster,Stevens,crimeLocation,sessionsPapers/17540116,guilty,NaN,NaN,NaN,1754
4,BAILEY,17540116,male,male,NaN,John,Thomas,NaN,theft,NaN,...,Allen,Fazakerley,NaN,NaN,sessionsPapers/17540116,notGuilty,NaN,NaN,NaN,1754
5,BAILEY,17540116,male,male,NaN,William,Thomas,NaN,theft,NaN,...,Derter,Wetworth,NaN,NaN,sessionsPapers/17540116,notGuilty,NaN,NaN,NaN,1754
6,BAILEY,17540116,male,male,male,William,Nicholas,Launcelot,theft,NaN,...,Ford,Healing,Hide,crimeLocation,sessionsPapers/17540116,guilty,NaN,NaN,NaN,1754
7,BAILEY,17540116,female,male,NaN,Anne,John,NaN,theft,NaN,...,Beezley,Jervas,NaN,NaN,sessionsPapers/17540116,guilty,NaN,NaN,NaN,1754
8,BAILEY,17540116,male,male,male,Robert,John,Abraham,theft,deception,...,Barber,Thorp,Julian,NaN,sessionsPapers/17540116,guilty,notGuilty,pleadedGuilty,noEvidence,1754
9,BAILEY,17540116,female,female,female,Elizabeth,Catherine,Catharine,theft,NaN,...,Eaton,Davis,Davis,crimeLocation,sessionsPapers/17540116,guilty,NaN,lesserOffence,NaN,1754


That's it! Now you know how to parse through XML files using XPath and web scrape using the `requests` library! 

## Bibliography

 - All files from Old Bailey API - https://www.oldbaileyonline.org/obapi/
 - ElementTree information adapted from Driscoll, Mike. (2013, April). Python 101 – Intro to XML Parsing with ElementTree.
 https://www.blog.pythonlibrary.org/2013/04/30/python-101-intro-to-xml-parsing-with-elementtree/

 - Web Scraping code adapted from MEDST-250 Notebook developed by Tejas Priyadarshan.
 https://github.com/ds-modules/MEDST-250/tree/master/04%20-%20XML_Day_1
 
 - Image source from https://www.researchgate.net/publication/257631377_Efficient_XML_Path_Filtering_Using_GPUs

----
Notebook developed by: Jason Jiang

Data Science Modules: http://data.berkeley.edu/education/modules